# Naive implementation of the problem solution

In this experiment we will consider a very straightforward solution to the problem. Namely, we will take the data and feed it to a convolutional neural network, which will act as a simple number crusher. I doubt that this implementation of the solution will be able to provide adequate accuracy. Therefore, the main goals at this stage will be:
- preparing a dataset
- augmentation
- setting up auxiliary utilities such as TensorBoard
- preparation of libraries for further use in subsequent experiments

As a result, we compare several convolutional neural network architectures. This will be our starting point for further research.

## Import necessary stuff

In [ ]:
import tensorflow as tf
from keras.layers import (Conv2D,
                          Dense,
                          Flatten,
                          MaxPool2D)
from keras.models import Model

from dataset import (ds, split_dataset, configure_for_performance,
                     apply_transformations)
from dataset.transformations import flatten_label, drop_alpha_channel
from dataset.utils import dummy_image
from models import build_head
from config import log_dir


## Prepare dataset

In [2]:
PROCESS_BATCH_SIZE=128
BATCH_SIZE = 64

train_ds, validation_ds = split_dataset(ds)
train_ds = apply_transformations(train_ds.repeat(2), batch=PROCESS_BATCH_SIZE)
train_ds = configure_for_performance(train_ds, batch_size=BATCH_SIZE)

validation_ds = validation_ds.map(flatten_label).map(drop_alpha_channel)
validation_ds = configure_for_performance(validation_ds,
                                          batch_size=BATCH_SIZE)

## Build and compile model, run on a dummy input and show summary

In [3]:
def build():
    input, x = build_head()

    x = Conv2D(64, 3, activation='relu')(x)
    x = MaxPool2D()(x)

    x = Conv2D(128, 3, activation='relu')(x)
    x = MaxPool2D()(x)

    x = Conv2D(256, 3, activation='relu')(x)
    x = MaxPool2D()(x)

    x = Conv2D(512, 3, activation='relu')(x)
    x = MaxPool2D()(x)

    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)

    output = Dense(4, activation=None)(x)

    return Model(inputs=input, outputs=output)

model = build()
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['accuracy'],
)

model(dummy_image(1))
model.summary()


2024-01-07 21:42:13.919963: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 344, 552, 3)]     0         
                                                                 
 base_preprocessing (BasePr  (None, 57, 92, 1)         0         
 eprocessing)                                                    
                                                                 
 conv2d (Conv2D)             (None, 55, 90, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 45, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 43, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 21, 128)       0     

## Train for a while and record results into TensorBoard

In [ ]:
EPOCHS = 100

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir + '/simple')

model.fit(train_ds,
          validation_data=validation_ds,
          epochs=EPOCHS,
          callbacks=[tensorboard_callback])